# Relationships in the GO
_Alex Warwick Vesztrocy, March 2016_

For some analyses, it is possible to only use the <code>is_a</code> definitions given in the Gene Ontology. 

However, it is important to remember that this isn't always the case. As such, <code>GOATOOLS</code> includes the option to load the relationship definitions also.

## Loading GO graph with the relationship tags
This is possible by using the <code>optional_attrs</code> argument, upon instantiating a <code>GODag</code>.

In [1]:
import os
from goatools.obo_parser import GODag

if not os.path.exists('go-basic.obo'):
    !wget http://geneontology.org/ontology/go-basic.obo
go = GODag('go-basic.obo', optional_attrs=['relationship'])

go-basic.obo: fmt(1.2) rel(2019-04-17) 47,398 GO Terms; optional_attrs(relationship)


## Viewing relationships in the GO graph

So now, when looking at an individual term (which has a relationship defined in the GO) these are listed in a nested manner. As an example, look at <code>GO:1901990</code> which has a single <code>regulates</code> relationship.

In [2]:
eg_term = go['GO:1901990']

In [3]:
eg_term

GOTerm('GO:1901990'):
  id:GO:1901990
  item_id:GO:1901990
  name:regulation of mitotic cell cycle phase transition
  namespace:biological_process
  _parents: 2 items
    GO:1901987
    GO:0007346
  parents: 2 items
    GO:1901987	level-06	depth-06	regulation of cell cycle phase transition [biological_process]
    GO:0007346	level-05	depth-05	regulation of mitotic cell cycle [biological_process]
  children: 6 items
    GO:1901991	level-07	depth-08	negative regulation of mitotic cell cycle phase transition [biological_process]
    GO:0010389	level-07	depth-08	regulation of G2/M transition of mitotic cell cycle [biological_process]
    GO:1901992	level-07	depth-08	positive regulation of mitotic cell cycle phase transition [biological_process]
    GO:0007096	level-07	depth-08	regulation of exit from mitosis [biological_process]
    GO:2000045	level-07	depth-08	regulation of G1/S transition of mitotic cell cycle [biological_process]
    GO:0030071	level-07	depth-10	regulation of mitotic me

These different relationship types are stored as a dictionary within the relationship attribute on a GO term.

In [4]:
print(eg_term.relationship.keys())

dict_keys(['regulates'])


In [5]:
print(eg_term.relationship['regulates'])

{GOTerm('GO:0044772'):
  id:GO:0044772
  item_id:GO:0044772
  name:mitotic cell cycle phase transition
  namespace:biological_process
  _parents: 2 items
    GO:0044770
    GO:1903047
  parents: 2 items
    GO:0044770	level-03	depth-03	cell cycle phase transition [biological_process]
    GO:1903047	level-03	depth-03	mitotic cell cycle process [biological_process]
  children: 4 items
    GO:0007091	level-05	depth-10	metaphase/anaphase transition of mitotic cell cycle [biological_process]
    GO:0010458	level-05	depth-05	exit from mitosis [biological_process]
    GO:0000082	level-05	depth-05	G1/S transition of mitotic cell cycle [biological_process]
    GO:0000086	level-05	depth-05	G2/M transition of mitotic cell cycle [biological_process]
  level:4
  depth:4
  is_obsolete:False
  alt_ids: 0 items
  relationship: 0 items
  relationship_rev: 3 items
    regulates: 1 items
      GO:1901990	level-06	depth-07	regulation of mitotic cell cycle phase transition [biological_process]
    negative

## Example use case
One example use case for the relationship terms, would be to look for all functions which regulate pseudohyphal growth (<code>GO:0007124</code>). That is:

> A pattern of cell growth that occurs in conditions of nitrogen limitation and abundant fermentable carbon source. Cells become elongated, switch to a unipolar budding pattern, remain physically attached to each other, and invade the growth substrate. 
>
> _Source: https://www.ebi.ac.uk/QuickGO/GTerm?id=GO:0007124#term=info&info=1_

In [6]:
term_of_interest = go['GO:0007124']

First, find the relationship types which contain "regulates":

In [7]:
regulates = frozenset([typedef 
                       for typedef in go.typedefs.keys() 
                       if 'regulates' in typedef])
print(regulates)

frozenset({'positively_regulates', 'regulates', 'negatively_regulates'})


Now, search through the terms in the tree for those with a relationship in this list and add them to a dictionary dependent on the type of regulation.

In [8]:
from collections import defaultdict

regulating_terms = defaultdict(list)

for t in go.values():
    if hasattr(t, 'relationship'):
        for typedef in regulates.intersection(t.relationship.keys()):
            if term_of_interest in t.relationship[typedef]:
                regulating_terms['{:s}d_by'.format(typedef[:-1])].append(t)

Now <code>regulating_terms</code> contains the GO terms which relate to regulating protein localisation to the nucleolus.

In [9]:
print('{:s} ({:s}) is:'.format(term_of_interest.name, term_of_interest.id))
for regulate_desc, goterms in regulating_terms.items():
    print('\n  - {:s}:'.format(regulate_desc))
    for goterm in goterms:
        print('    -- {:s} {:s}'.format(goterm.id, goterm.name))
        for gochild in goterm.children:
            print('    -- {:s} {:s}'.format(gochild.id, gochild.name))

pseudohyphal growth (GO:0007124) is:

  - regulated_by:
    -- GO:2000220 regulation of pseudohyphal growth
    -- GO:2000222 positive regulation of pseudohyphal growth
    -- GO:2000221 negative regulation of pseudohyphal growth

  - negatively_regulated_by:
    -- GO:2000221 negative regulation of pseudohyphal growth
    -- GO:1900462 negative regulation of pseudohyphal growth by negative regulation of transcription from RNA polymerase II promoter
    -- GO:0100042 negative regulation of pseudohyphal growth by transcription from RNA polymerase II promoter

  - positively_regulated_by:
    -- GO:2000222 positive regulation of pseudohyphal growth
    -- GO:0100041 positive regulation of pseudohyphal growth by transcription from RNA polymerase II promoter
    -- GO:1900461 positive regulation of pseudohyphal growth by positive regulation of transcription from RNA polymerase II promoter
